使用正弦函数和一些可加性噪声来生成序列数据， 时间步为1,2,...1000。

In [ ]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l

T = 1000
time = torch.arange(1, T+1, dtype=torch.float32)
x = torch.sin(time * 0.01) + torch.normal(0, 0.2, (T,))
d2l.plot(time, [x], 'time', 'x', xlim=[1, 1000], figsize=(6, 3))

# 我们将数据映射为数据对$y_t = x_t$ 和$\mathbf{x}t = [x{t-\tau}, \ldots, x_{t-1}]$。

In [15]:
tau = 4
features = torch.zeros((T-tau, tau))  # T-tau样本数，tau特征数
for i in range(tau):
    features[:, i] = x[i: T-tau+i]  # 每tau个数据作为一组输入
labels = x[tau:].reshape((-1, 1))

batch_size, n_train = 16, 600
train_iter = d2l.load_array((features[:n_train], labels[:n_train]), batch_size, is_train=True)